## Extracción y transformación de datos con Spark 2.4
### Origen de datos
Los datos representan los accidentes de tráfico por día y distrito
que se han producido en la ciudad de Madrid entre 2010 y 2018

El fichero origen es un fichero csv por año tiene los datos detallados
por cada uno de los intervinientes en el accidente: Conductor, pasajero,
testito, peatón, ...

La primera parte del fichero está referida al accidente y se repite
por cada uno de los intervinientes.

### La parte del registro referida al accidente tiene los campos:
 - 'FECHA'
 - 'RANGO HORARIO'
 - 'DIA SEMANA'
 - 'DISTRITO'
 - 'LUGAR ACCIDENTE'
 - 'Nº: string'
 - 'Nº PARTE'     **Id.accidente, es el mismo para varios regs.**
 - 'CPFA Granizo'
 - 'CPFA Hielo'
 - 'CPFA Lluvia'
 - 'CPFA Niebla'
 - 'CPFA Seco'
 - 'CPFA Nieve'
 - 'CPSV Mojada'
 - 'CPSV Aceite'
 - 'CPSV Barro'
 - 'CPSV Grava Suelta'
 - 'CPSV Hielo'
 - 'CPSV Seca Y Limpia'
 - '* Nº VICTIMAS' **Total de víctimas por Nº de parte**
 - 'TIPO ACCIDENTE'
 - 'Tipo Vehiculo'
 
### La parte del registro correspondiente a cada persona:
 - 'TIPO PERSONA'
 - 'SEXO'
 - 'LESIVIDAD'
 - 'Tramo Edad'


In [1]:
import pyspark
from pyspark.sql import SparkSession
app_name = "bubbly"
master = "local[*]"
spark = (SparkSession.builder
    .master(master)
    .config("spark.driver.cores", 1)
    .appName(app_name)
    .getOrCreate() )
sc = spark.sparkContext
print ('SparkContext created')

SparkContext created


In [2]:
# Cargamos el fichero desde hdfs
accidenteData = spark.read.csv ('hdfs://localhost:9000/user/ubuntu/accidentes/2018.csv',header=True)
#accidenteData = spark.read.csv ('file:///home/ubuntu/Downloads/2018_Accidentalidad.csv',header=True)


In [3]:
import pyspark.sql.functions as func

# Reducimos las columnas que tenemos que utilizar mediante la Select
# y aplicamos las funciones necesarias a los datos
accRed=accidenteData.select(
#        func.to_date(accidenteData.FECHA, 'dd/MM/yyyy HH:mm:ss').alias('fecha'), \
        func.to_date(accidenteData.FECHA, 'dd/MM/yyyy').alias('fecha'), \
        accidenteData.DISTRITO.alias('distrito'), \
        accidenteData["Nº PARTE"].alias('idAccidente'), \
        func.substring(accidenteData.LESIVIDAD,1,2).alias('lesividad'), \
)

In [4]:
#Las columnas del Dataframe accRed quedan así
accRed.printSchema()

root
 |-- fecha: date (nullable = true)
 |-- distrito: string (nullable = true)
 |-- idAccidente: string (nullable = true)
 |-- lesividad: string (nullable = true)



In [5]:
# Eliminamos las filas de las personas ilesas
# Agrupamos por fecha, distrito, accidente y lesividad para contar 
# los registros y tener el total de afectados.
accGrouped=accRed.filter(accRed.lesividad != "IL")\
                .groupBy('fecha','distrito','idAccidente','lesividad')\
                .agg(func.count('lesividad').alias('victimas'))\
                .sort ('fecha','distrito','idAccidente','lesividad')

In [6]:
#Las columnas del Dataframe accRed quedan así
accGrouped.printSchema()

root
 |-- fecha: date (nullable = true)
 |-- distrito: string (nullable = true)
 |-- idAccidente: string (nullable = true)
 |-- lesividad: string (nullable = true)
 |-- victimas: long (nullable = false)



#### Por comodidad para poder guardar todos los datos en un único fichero utilizamos la librería pandas.


In [7]:
accGrouped.toPandas().to_csv('/home/ubuntu/Downloads/victimas_por_fecha_distrito_accidente_lesion.csv',sep=';')